   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.1/606.1 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.66.5
    Uninstalling openai-1.66.5:
      Successfully uninstalled openai-1.66.5


In [2]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

# Install LangChain dependencies
!pip install langchain langchain-community langchain-core

# Install HuggingFace Transformers and related dependencies
!pip install transformers

# Install BeautifulSoup4 for HTML parsing
!pip install beautifulsoup4

# Install Chroma for vector store
!pip install chromadb

# Install LangGraph for LangChain and HuggingFace integration
!pip install langgraph

# Install tiktoken for tokenization
!pip install tiktoken

# Install numpy for numerical operations (cosine similarity)
!pip install numpy

# Install sentence-transformers for the pre-trained transformer models (optional for embedding generation)
!pip install sentence-transformers


!pip install faiss-cpu
!pip install faiss-gpu

# Install necessary libraries
!pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all fastembed langchain-groq



ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [4]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'

In [17]:

# Import necessary libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_groq import ChatGroq
from langchain.schema import BaseMessage, HumanMessage
import time

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


# Initialize the embedding model
embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

# Set up the LLM with Groq's Chat API
llm = ChatGroq(temperature=0,
               model_name="Llama3-8b-8192",
               api_key="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")

# Example usage of Groq's LLM
message = HumanMessage(content="Hello, how are you?")
response = llm([message])
print(response)

#### DOCUMENT LOADING AND EMBEDDING ####

# Step 1: Load the blog content from a URL using WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# Load documents from the URL
docs = loader.load()

# Step 2: Split the documents into smaller chunks for better processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Step 3: Create the vector store (Chroma) with FastEmbed embeddings
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embed_model,  # Using FastEmbedEmbeddings for embedding
    collection_name="local-rag"
)

# Step 4: Set up the retriever to query the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

#### ROUTER CHAIN SETUP ####

# Step 5: Define the prompt template for routing the question
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a
    user question to a vectorstore or web search. Use the vectorstore for questions on LLM  agents,
    prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search'
    or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and
    no premable or explanation. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

# Step 6: Initialize the Router Chain
start = time.time()
question_router = prompt | llm | JsonOutputParser()

# Step 7: Query the router chain for a sample question
question = "llm agent memory"
print(question_router.invoke({"question": question}))
end = time.time()

# Print the time taken to generate the response
print(f"The time required to generate response by Router Chain in seconds:{end - start}")


content="I'm just a language model, I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you may have! How can I assist you today?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 16, 'total_tokens': 61, 'completion_time': 0.0375, 'prompt_time': 0.002412121, 'queue_time': 0.022125588, 'total_time': 0.039912121}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None} id='run-a7e61f92-745e-451f-a20c-483800fde3a0-0' usage_metadata={'input_tokens': 16, 'output_tokens': 45, 'total_tokens': 61}
{'datasource': 'vectorstore'}
The time required to generate response by Router Chain in seconds:0.19173288345336914


In [18]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
# Question
rag_chain.invoke("What is Task Decomposition?")

'{\n"datasource": "web_search"\n}'

In [ ]:
-------------------------------------------

In [8]:
# Import necessary libraries
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.schema import BaseMessage, HumanMessage
import time

# Initialize the embedding model
embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

# Set up the LLM with Groq's Chat API
llm = ChatGroq(temperature=0,
               model_name="Llama3-8b-8192",
               api_key="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")

# Example usage of Groq's LLM
message = HumanMessage(content="Hello, how are you?")
response = llm([message])
print(response)

#### DOCUMENT LOADING AND EMBEDDING ####

# Step 1: Load the blog content from a URL using WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# Load documents from the URL
docs = loader.load()

# Step 2: Split the documents into smaller chunks for better processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Step 3: Create the vector store (Chroma) with FastEmbed embeddings
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embed_model,  # Using FastEmbedEmbeddings for embedding
    collection_name="local-rag"
)

# Step 4: Set up the retriever to query the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

#### ROUTER CHAIN SETUP ####

# Step 5: Define the prompt template for routing the question
prompt = hub.pull("rlm/rag-prompt")

# Step 6: Initialize the Router Chain
start = time.time()
question_router = prompt | llm | StrOutputParser()

# Step 7: Query the router chain for a sample question
question = "llm agent memory"
# Step 8: Format the retrieved documents into a suitable format for the LLM
def format_docs(docs):
    """Formats the retrieved documents into a string."""
    return "\n\n".join(doc.page_content for doc in docs)

# Make sure to format the documents (context) properly
formatted_docs = format_docs(retriever.get_relevant_documents(question))  # Get context from retriever

# Invoke the router chain with both 'context' and 'question'
response = question_router.invoke({"context": formatted_docs, "question": question})
end = time.time()

# Print the time taken to generate the response
print(f"Response: {response}")
print(f"The time required to generate response by Router Chain in seconds: {end - start}")

#### RETRIEVAL AND GENERATION CHAIN SETUP ####


# Step 9: Set up the RAG chain by combining retrieval, prompt, LLM, and output parser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()  # Clean the output for the final response
)

# Step 10: Invoke the RAG chain to answer the question based on the context
question = "What is Task Decomposition?"
response = rag_chain.invoke(question)
print(f"RAG Chain Response: {response}")


content="I'm just a language model, I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you may have! How can I assist you today?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 16, 'total_tokens': 61, 'completion_time': 0.0375, 'prompt_time': 0.002409923, 'queue_time': 0.017351896999999998, 'total_time': 0.039909923}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None} id='run-3532c447-bb24-4d8f-9921-b2f2e19a037a-0' usage_metadata={'input_tokens': 16, 'output_tokens': 45, 'total_tokens': 61}


<ipython-input-8-977bccf69924>:73: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  formatted_docs = format_docs(retriever.get_relevant_documents(question))  # Get context from retriever


Response: According to the context, the LLM-powered autonomous agent system has a memory component that allows the agent to reflect on past actions, learn from mistakes, and refine them for future steps, thereby improving the quality of final results.
The time required to generate response by Router Chain in seconds: 0.41331982612609863
RAG Chain Response: Task Decomposition is a technique that breaks down complex tasks into smaller, simpler steps, allowing the model to plan ahead and utilize more test-time computation. This process, known as Chain of Thought (CoT), enhances model performance on complex tasks by transforming big tasks into manageable tasks.


In [ ]:
----------------------working full code from here------------------------

In [9]:
# Import necessary libraries
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.schema import BaseMessage, HumanMessage
import time
import tiktoken
import numpy as np

# Initialize the embedding model
embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

# Set up the LLM with Groq's Chat API
llm = ChatGroq(temperature=0,
               model_name="Llama3-8b-8192",
               api_key="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")

# Example usage of Groq's LLM
message = HumanMessage(content="Hello, how are you?")
response = llm([message])
print(response)

#### DOCUMENT LOADING AND EMBEDDING ####

# Step 1: Load the blog content from a URL using WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# Load documents from the URL
docs = loader.load()

# Step 2: Split the documents into smaller chunks for better processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Step 3: Create the vector store (Chroma) with FastEmbed embeddings
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embed_model,  # Using FastEmbedEmbeddings for embedding
    collection_name="local-rag"
)

# Step 4: Set up the retriever to query the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

#### ROUTER CHAIN SETUP ####

# Step 5: Define the prompt template for routing the question
prompt = hub.pull("rlm/rag-prompt")

# Step 6: Initialize the Router Chain
start = time.time()
question_router = prompt | llm | StrOutputParser()

# Step 7: Query the router chain for a sample question
question = "llm agent memory"

# Step 8: Format the retrieved documents into a suitable format for the LLM
def format_docs(docs):
    """Formats the retrieved documents into a string."""
    return "\n\n".join(doc.page_content for doc in docs)

# Make sure to format the documents (context) properly
formatted_docs = format_docs(retriever.get_relevant_documents(question))  # Get context from retriever

# Invoke the router chain with both 'context' and 'question'
response = question_router.invoke({"context": formatted_docs, "question": question})
end = time.time()

# Print the time taken to generate the response
print(f"Response: {response}")
print(f"The time required to generate response by Router Chain in seconds: {end - start}")

#### RETRIEVAL AND GENERATION CHAIN SETUP ####

# Step 9: Set up the RAG chain by combining retrieval, prompt, LLM, and output parser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()  # Clean the output for the final response
)

# Step 10: Invoke the RAG chain to answer the question based on the context
question = "What is Task Decomposition?"
response = rag_chain.invoke(question)
print(f"RAG Chain Response: {response}")

#### COSINE SIMILARITY CALCULATION ####

# Example documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

# Function to calculate the number of tokens in a string
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Get number of tokens in question
num_tokens_from_string(question, "cl100k_base")

# Embed the query and document
query_result = embed_model.embed_query(question)
document_result = embed_model.embed_query(document)

# Cosine similarity function
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# Calculate similarity between the query and document
similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)


content="I'm just a language model, I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you may have! How can I assist you today?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 16, 'total_tokens': 61, 'completion_time': 0.0375, 'prompt_time': 0.003138088, 'queue_time': 0.020484301, 'total_time': 0.040638088}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None} id='run-27f46bea-bde4-4ef3-8edf-8f361c935433-0' usage_metadata={'input_tokens': 16, 'output_tokens': 45, 'total_tokens': 61}
Response: According to the context, the LLM-powered autonomous agent system has a memory component that allows the agent to reflect on past actions, learn from mistakes, and refine them for future steps, thereby improving the quality of final results.
The time required to generate response by Router Chain in seconds: 0.

In [10]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [11]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [12]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
# Step 3: Create the vector store (Chroma) with FastEmbed embeddings
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embed_model,  # Using FastEmbedEmbeddings for embedding
    collection_name="local-rag"
)

# Step 4: Set up the retriever to query the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [13]:
docs = retriever.get_relevant_documents("What is Task Decomposition?")

In [14]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage
import time

# Define the prompt template manually
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# Create a prompt template using the defined string
prompt = ChatPromptTemplate.from_template(template)

# Set up the LLM with Groq's Chat API
llm = ChatGroq(temperature=0,
               model_name="Llama3-8b-8192",
               api_key="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")

# Chain the prompt and the LLM model
chain = prompt | llm

# Example documents and query
docs = "This is the context of the Task Decomposition process."
question = "What is Task Decomposition?"

# Run the chain
response = chain.invoke({"context": docs, "question": question})
print(response)


content='Task Decomposition is the process of breaking down a large, complex task or project into smaller, more manageable tasks or subtasks.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 39, 'total_tokens': 66, 'completion_time': 0.0225, 'prompt_time': 0.005401893, 'queue_time': 0.015970716000000003, 'total_time': 0.027901893}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None} id='run-093d001a-10e4-4861-af33-b83cdbced188-0' usage_metadata={'input_tokens': 39, 'output_tokens': 27, 'total_tokens': 66}


In [15]:
!pip install llama_index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 19.6 MB/s eta 0:00:00


In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'According to the provided context, Task Decomposition is the process of breaking down a complicated task into smaller and simpler steps. This is achieved through the use of Chain of Thought (CoT), a prompting technique that instructs the model to "think step by step" to utilize more test-time computation.'